# AI Lesson 04b: GROUP BY for Machine Learning

**Course:** Applications of Artificial Intelligence  
**Focus:** Grouping and Aggregating Data for ML  

---

## Learning Objectives

By the end of this short lesson, you will:
- Understand what `GROUP BY` does and why it's crucial for ML
- Calculate statistics per category (team, season, etc.)
- Use `HAVING` to filter grouped results
- Prepare aggregated datasets for machine learning

**Why this matters for ML:**  
In our next lesson (ai05), we'll build a **Random Forest model** to predict NBA game winners. To do that, we need to calculate **team season averages** using the exact GROUP BY skills you'll learn today.

---

## Setup

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('nba_5seasons (2).db')
print("✅ Connected to NBA database")

✅ Connected to NBA database


---

## Part 1: The Problem GROUP BY Solves

### Without GROUP BY

Let's say we want to know the **average points per game for each team** in the 2024-25 season.

**Without GROUP BY**, we'd have to:
1. Filter for each team individually
2. Calculate the average
3. Repeat 30 times (once per team!)

That's tedious and inefficient. Let's see this problem:

In [4]:
# The hard way: Calculate average points for ONE team
query = """
SELECT AVG(pts) as avg_points
FROM team_game_stats
WHERE season = '2024-25' 
  AND team_id = 1610612739
"""

result = pd.read_sql(query, conn)
print("Average points for team 1610612739 (Cavaliers):")
display(result)

print("\n❌ Problem: We'd have to write 30 different queries for all teams!")

Average points for team 1610612739 (Cavaliers):


,avg_points
0,121.939024



❌ Problem: We'd have to write 30 different queries for all teams!


### With GROUP BY 

**GROUP BY** splits our data into groups (one per team) and calculates statistics for each group **automatically**.

Think of it like this:
```
All games → Split by team_id → Calculate AVG(pts) for each team → Done!
```

In [5]:
# The smart way: Calculate average points for ALL teams at once
query = """
SELECT 
    team_id,
    AVG(pts) as avg_points
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_points DESC
"""

result = pd.read_sql(query, conn)
print("✅ Average points for ALL teams in one query!")
display(result.head(10))

✅ Average points for ALL teams in one query!


,team_id,avg_points
0,1610612739,121.939024
1,1610612763,121.707317
2,1610612743,120.756098
3,1610612760,120.500000
4,1610612737,118.182927
5,1610612741,117.804878
6,1610612754,117.365854
7,1610612738,116.268293
8,1610612752,115.780488
9,1610612758,115.731707


### Key Insight

**GROUP BY pattern:**
```sql
SELECT 
    category_column,           -- What to group by
    AGGREGATE_FUNCTION(value)  -- What to calculate
FROM table
GROUP BY category_column
```

**Rules:**
1. Columns in SELECT must EITHER be:
   - In the GROUP BY clause, OR
   - Inside an aggregate function (AVG, SUM, COUNT, MAX, MIN)
2. GROUP BY creates one row per unique value in the grouping column

---

## Part 2: Practice GROUP BY

### Task 1: Count Games Per Season

Write a query to count how many games were played in EACH season.

**Hints:**
- Use `COUNT(*)` to count rows
- Group by the `season` column
- Order by season

In [8]:
query = """
SELECT COUNT(*), season
FROM player_season_stats
GROUP BY season
ORDER BY season
"""

result = pd.read_sql(query, conn)
display(result)

,COUNT(*),season
0,605,2021-22
1,539,2022-23
2,572,2023-24
3,569,2024-25
4,506,2025-26


**Expected:** One row per season with game count

### Task 2: Team Season Averages

For the 2024-25 season, calculate average points, assists, and rebounds for EACH team.

**Hints:**
- Filter with WHERE before grouping
- Use AVG() for each stat
- Order by avg_points descending

### Task 3: Count Wins Per Team

For the 2024-25 season, count how many wins each team has.

**Hints:**
- WHERE season = '2024-25' AND wl = 'W'
- COUNT(*) will count wins
- GROUP BY team_id
- ORDER BY wins descending

In [23]:
query = """
SELECT team_id, COUNT(wl), season
FROM team_game_stats
WHERE season = '2024-25' AND wl = 'W'
GROUP BY team_id
ORDER BY wl desc
"""

result = pd.read_sql(query, conn)
display(result.head(10))

,team_id,COUNT(wl),season
0,1610612766,19,2024-25
1,1610612765,44,2024-25
2,1610612764,18,2024-25
3,1610612763,48,2024-25
4,1610612762,17,2024-25
5,1610612761,30,2024-25
6,1610612760,68,2024-25
7,1610612759,34,2024-25
8,1610612758,40,2024-25
9,1610612757,36,2024-25


---

## Part 3: Multiple Aggregations

### Task 4: Complete Team Stats

For each team in 2024-25, calculate:
- Game count
- Average points
- Highest score (MAX)
- Lowest score (MIN)

**All in one query!**

In [25]:
query = """
SELECT team_id, gp, AVG(pts), MAX(pts), MIN(pts), season
FROM player_season_stats
WHERE season = '2024-25'
GROUP BY team_id
"""

result = pd.read_sql(query, conn)
display(result.head(10))

,team_id,gp,AVG(pts),MAX(pts),MIN(pts),season
0,1610612737,17,538.277778,1841.0,23.0,2024-25
1,1610612738,3,561.823529,1932.0,7.0,2024-25
2,1610612739,12,565.333333,1701.0,12.0,2024-25
3,1610612740,19,414.095238,1183.0,28.0,2024-25
4,1610612741,4,460.950000,1509.0,8.0,2024-25
5,1610612742,16,524.631579,1261.0,24.0,2024-25
6,1610612743,13,582.470588,2071.0,24.0,2024-25
7,1610612744,2,487.411765,1718.0,2.0,2024-25
8,1610612745,9,528.444444,1723.0,14.0,2024-25
9,1610612746,1,503.000000,1802.0,5.0,2024-25


**Notice:** You can use as many aggregate functions as you want in one query!

---

## Part 4: HAVING - Filter Groups

### The Problem

What if we only want teams with **50+ wins**?

**We can't use WHERE** because WHERE filters individual rows BEFORE grouping.

**We need HAVING** to filter groups AFTER aggregation.

### WHERE vs HAVING

| Clause | When | Filters | Example |
|--------|------|---------|----------|
| WHERE | Before grouping | Individual rows | `WHERE season = '2024-25'` |
| HAVING | After grouping | Aggregated groups | `HAVING COUNT(*) >= 50` |

**Query order:**
```sql
SELECT ...
FROM ...
WHERE ...      -- Filter rows first
GROUP BY ...   -- Then group
HAVING ...     -- Then filter groups
ORDER BY ...   -- Finally sort
```

### Example: Elite Teams (50+ Wins)

In [26]:
query = """
SELECT 
    team_id,
    COUNT(*) as wins
FROM team_game_stats
WHERE season = '2024-25' 
  AND wl = 'W'
GROUP BY team_id
HAVING COUNT(*) >= 50    -- Filter groups with 50+ wins
ORDER BY wins DESC
"""

result = pd.read_sql(query, conn)
print("Elite teams with 50+ wins:")
display(result)

Elite teams with 50+ wins:


,team_id,wins
0,1610612760,68
1,1610612739,64
2,1610612738,61
3,1610612745,52
4,1610612752,51
5,1610612754,50
6,1610612747,50
7,1610612746,50
8,1610612743,50


### Task 5: High-Scoring Teams

Find all teams averaging 110+ points per game in 2024-25.

**Hints:**
- GROUP BY team_id
- Use HAVING with AVG(pts)
- Include the avg_points in your SELECT

In [29]:
query = """
SELECT team_id, AVG(pts), season
FROM player_season_stats
WHERE season = '2024-25'
GROUP BY team_id
HAVING pts >= 110
"""

result = pd.read_sql(query, conn)
display(result)

,team_id,AVG(pts),season
0,1610612737,538.277778,2024-25
1,1610612738,561.823529,2024-25
2,1610612740,414.095238,2024-25
3,1610612741,460.950000,2024-25
4,1610612742,524.631579,2024-25
5,1610612743,582.470588,2024-25
6,1610612744,487.411765,2024-25
7,1610612745,528.444444,2024-25
8,1610612746,503.000000,2024-25
9,1610612747,415.476190,2024-25


---

## Part 5: Preview - GROUP BY in Machine Learning

### Real Use Case: Random Forest Prep

In **ai05 (next lesson)**, we'll build a Random Forest model to predict game winners.

**The query we'll use:**

In [28]:
# This is the EXACT query from ai05 - GROUP BY in action!
team_avg_query = """
SELECT 
    team_id,
    AVG(pts) as avg_pts,
    AVG(ast) as avg_ast,
    AVG(reb) as avg_reb,
    AVG(stl) as avg_stl,
    AVG(blk) as avg_blk,
    AVG(tov) as avg_tov,
    AVG(fg_pct) as avg_fg_pct,
    COUNT(*) as games_played
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_pts DESC
"""

team_averages = pd.read_sql(team_avg_query, conn)
print("Team Season Averages (for ML prediction):")
display(team_averages.head(10))

DatabaseError: Execution failed on sql '
SELECT 
    team_id,
    AVG(pts) as avg_pts,
    AVG(ast) as avg_ast,
    AVG(reb) as avg_reb,
    AVG(stl) as avg_stl,
    AVG(blk) as avg_blk,
    AVG(tov) as avg_tov,
    AVG(fg_pct) as avg_fg_pct,
    COUNT(*) as games_played
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_pts DESC
': no such column: fg_pct

### How This Powers ML

In ai05, we'll:
1. Pull these team averages using GROUP BY ✅ (what you just learned!)
2. Calculate differentials (Team A avg - Team B avg)
3. Feed differentials into Random Forest model
4. Predict: "Will the home team win?"

**Without GROUP BY, this wouldn't be possible!**

---

## Part 6: Challenge Tasks

### Challenge 1: Win Percentage

Calculate each team's win percentage for 2024-25.

**Formula:** `(wins / total_games) * 100`

**Hints:**
- Count total games with `COUNT(*)`
- Count wins with `SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END)`
- Use `* 100.0` to get percentage
- Round with `ROUND(value, 1)` for one decimal place

In [ ]:
query = """
-- YOUR QUERY HERE

"""

result = pd.read_sql(query, conn)
display(result.head(10))

### Challenge 2: Season-by-Season Scoring Trends

Calculate average points for EACH season to see if scoring is increasing over time.

**Hint:** GROUP BY season, ORDER BY season

In [ ]:
query = """
-- YOUR QUERY HERE

"""

result = pd.read_sql(query, conn)
display(result)

**Question:** Is scoring going up or down over the years?

---

## Cleanup

In [ ]:
conn.close()
print("✅ Database connection closed")

---

## Summary

### What You Learned

✅ **GROUP BY** - Split data by category and aggregate  
✅ **Aggregate functions** - COUNT, SUM, AVG, MAX, MIN  
✅ **HAVING** - Filter groups after aggregation  
✅ **WHERE vs HAVING** - When to use each  
✅ **ML connection** - How GROUP BY prepares data for models  

### Key Patterns

```sql
-- Basic grouping
SELECT category, AVG(value)
FROM table
GROUP BY category

-- With filtering
SELECT category, COUNT(*)
FROM table
WHERE condition        -- Filter rows BEFORE grouping
GROUP BY category
HAVING COUNT(*) > 50   -- Filter groups AFTER aggregating
ORDER BY COUNT(*) DESC
```

### Complete Query Order

```sql
SELECT      -- What columns/calculations
FROM        -- Which table
WHERE       -- Filter individual rows
GROUP BY    -- Split into groups
HAVING      -- Filter groups
ORDER BY    -- Sort results
LIMIT       -- Limit results
```

---

## Next Up: AI05 - Random Forest 

You're now ready for **ai05** where you'll:
1. Pull live NBA data from an API
2. Use GROUP BY to calculate team averages (exactly what you learned today!)
3. Build a Random Forest model
4. Predict NBA game winners

**The GROUP BY skills you learned today are essential for that lesson!**